In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import tensorflow as tf
from transformers import TFAutoModel, AutoTokenizer
import pandas as pd

In [4]:
data = pd.read_excel('C:\\Users\\kordz\\OneDrive\\Desktop\\Final Project\\Bert_Data.xlsx')

In [5]:
data

,word,category
0,9 am to 11:30 am,breakfast
1,6:30-8 pm,breakfast
2,4-9 pm,breakfast
3,4-6 pm,breakfast
4,8:30-11:30,breakfast
...,...,...
950,Enjoying the feast of flavors!,comments
951,Delighting in gourmet goodness!,comments
952,Feeding my passion for food!,comments
953,Tasting the sweetness of life!,comments


In [ ]:
model = TFAutoModel.from_pretrained("bert-base-uncased")
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [ ]:
def tokenize_text(texts):
    return tokenizer(texts, padding=True, truncation=True, return_tensors='tf')

In [ ]:
data['word'] = data['word'].astype(str)
inputs = tokenize_text(data['word'].tolist())

In [ ]:
label_map = {label: idx for idx, label in enumerate(data['category'].unique())}
data['label'] = data['category'].map(label_map)

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((dict(inputs), data['label'].tolist()))

In [ ]:
BATCH_SIZE = 64
dataset = dataset.shuffle(len(data)).batch(BATCH_SIZE)

In [ ]:
class BERTForClassification(tf.keras.Model):

    def __init__(self, bert_model, num_classes):
        super().__init__()
        self.bert = bert_model
        self.fc = tf.keras.layers.Dense(num_classes, activation='softmax')

    def call(self, inputs):
        x = self.bert(inputs)[1]
        return self.fc(x)

In [ ]:
num_classes = len(label_map)
classifier = BERTForClassification(model, num_classes=num_classes)

In [ ]:
classifier.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

history = classifier.fit(
    dataset,
    epochs=5
)

Epoch 1/5
15/15 [==============================] - 251s 13s/step - loss: 1.6485 - accuracy: 0.4346
Epoch 2/5
15/15 [==============================] - 191s 13s/step - loss: 1.1848 - accuracy: 0.6461
Epoch 3/5
15/15 [==============================] - 191s 13s/step - loss: 0.8135 - accuracy: 0.7654
Epoch 4/5
15/15 [==============================] - 197s 13s/step - loss: 0.5456 - accuracy: 0.8953
Epoch 5/5
15/15 [==============================] - 192s 13s/step - loss: 0.3414 - accuracy: 0.9581


In [ ]:
 new_data = ['6:00-9:00 pm']

# Tokenize input data
tokenized_input = tokenizer(new_data, padding=True, truncation=True, return_tensors="tf")

# Make predictions using the BERT model
outputs = model(tokenized_input)

# Assuming classification task, take logits from the output
logits = outputs.last_hidden_state


num_classes = 6
classifier_layer = tf.keras.layers.Dense(num_classes, activation='softmax')
predictions = classifier_layer(logits[:, 0, :])

predicted_categories = tf.argmax(predictions, axis=-1)

# Map indices to actual category labels (if needed)
# Replace this with your actual mapping of indices to labels
label_map = {0: 'breakfast', 1: 'lunch', 2: 'dinner', 3: 'snacks', 4: 'drinks', 5: 'not_category'}

# Map predicted indices to labels
predicted_labels = [label_map[idx.numpy()] for idx in predicted_categories]

# Print or use predicted labels as needed
print("Predicted Labels:", predicted_labels)

Predicted Labels: ['drinks']


# This model has quite poor performance because having no proper data and data which is used here is generated by chat gpt.